# Context Compression with Amazon Bedrock

**Context Compression** = Reducing context size while preserving relevant information for LLM generation.

Based on:
- [How to Fix Your Context](https://www.dbreunig.com/2025/06/26/how-to-fix-your-context.html) by Drew Breunig
- [LangChain Context Engineering](https://github.com/langchain-ai/context_engineering) - Compress Context notebook
- [Provence: Efficient and Robust Context Pruning](https://arxiv.org/abs/2501.16214) (ICLR 2025)

## Context Compression Techniques

| Technique | Description | Best For |
|-----------|-------------|----------|
| **Pruning** | Remove irrelevant sentences | Document QA, RAG |
| **Summarization** | Condense into shorter form | Long conversations, histories |
| **Extraction** | Pull out key facts/entities | Structured data extraction |
| **Reranking** | Reorder by relevance | Multi-document retrieval |

## Why Compress Context?

Long contexts fail in several ways:
- **Context Confusion**: Superfluous info leads to low-quality responses
- **Context Distraction**: Model over-focuses on context, neglects training
- **Context Poisoning**: Errors propagate through repeated references
- **Context Clash**: New info conflicts with existing prompt content

Compression addresses these by keeping only what's relevant and reducing token count.

In [ ]:
!pip install boto3 transformers torch nltk -q

In [ ]:
import boto3
import json
import time
import nltk
from typing import List, Dict, Tuple

nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')
print('✓ Bedrock client ready')

## 1. Sample Document (Wikipedia-style)

A long document with mixed relevant/irrelevant content for our question.

In [ ]:
SAMPLE_DOCUMENT = """
Amazon Web Services (AWS) is a subsidiary of Amazon that provides on-demand cloud computing platforms and APIs to individuals, companies, and governments, on a metered, pay-as-you-go basis. AWS was launched in 2006 and has since become the world's most comprehensive and broadly adopted cloud platform.

The company's headquarters are located in Seattle, Washington. AWS employs over 100,000 people worldwide. The CEO of AWS is Matt Garman, who took over the role in 2024.

Amazon Bedrock is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Mistral AI, Stability AI, and Amazon through a single API. Amazon Bedrock makes it easy to build and scale generative AI applications with security, privacy, and responsible AI.

The AWS re:Invent conference is held annually in Las Vegas, Nevada. The conference typically attracts over 50,000 attendees. The first re:Invent was held in 2012.

With Amazon Bedrock, you can easily experiment with and evaluate top FMs for your use case, privately customize them with your data using techniques such as fine-tuning and Retrieval Augmented Generation (RAG), and build agents that execute tasks using your enterprise systems and data sources.

AWS has data centers in 33 geographic regions around the world. Each region consists of multiple Availability Zones. The company continues to expand its global infrastructure.

Bedrock supports several model families including Claude (Anthropic), Titan (Amazon), Llama (Meta), Command (Cohere), and Jurassic (AI21 Labs). The service provides serverless access to these models, meaning you don't need to manage any infrastructure.

Amazon's founder Jeff Bezos started the company in 1994 as an online bookstore. The company went public in 1997. Today, Amazon is one of the world's most valuable companies.

Key features of Amazon Bedrock include: model customization through fine-tuning, knowledge bases for RAG, agents for task automation, guardrails for responsible AI, and model evaluation capabilities. The service integrates seamlessly with other AWS services.

The Amazon rainforest, also known as Amazonia, is a moist broadleaf tropical rainforest in the Amazon biome. It covers most of the Amazon basin of South America. This is unrelated to Amazon the company.
"""

print(f"Document length: {len(SAMPLE_DOCUMENT)} characters")
print(f"Sentences: {len(nltk.sent_tokenize(SAMPLE_DOCUMENT))}")

## 2. Method 1: LLM-Based Pruning

Use an LLM to identify and extract only relevant sentences.
This is the approach described in the Provence paper's silver label generation.

In [ ]:
def llm_prune_context(question: str, document: str, model_id: str = 'anthropic.claude-3-5-sonnet-20241022-v2:0') -> Dict:
    """Use LLM to prune irrelevant sentences from context."""
    sentences = nltk.sent_tokenize(document)
    numbered = '\n'.join([f'[{i}] {s}' for i, s in enumerate(sentences)])
    
    prompt = f"""Given the question and numbered sentences, return ONLY the sentence numbers that are relevant to answering the question.

Question: {question}

Sentences:
{numbered}

Return JSON array of relevant sentence numbers only. Example: [0, 2, 5]
Relevant sentences:"""
    
    start = time.time()
    response = bedrock.converse(
        modelId=model_id,
        messages=[{'role': 'user', 'content': [{'text': prompt}]}],
        inferenceConfig={'temperature': 0, 'maxTokens': 500}
    )
    latency = time.time() - start
    
    content = response['output']['message']['content'][0]['text']
    if '```' in content:
        content = content.split('```')[1].replace('json', '').strip()
    
    try:
        indices = json.loads(content)
        pruned = ' '.join([sentences[i] for i in indices if i < len(sentences)])
    except:
        pruned = document
        indices = list(range(len(sentences)))
    
    return {
        'original_length': len(document),
        'pruned_length': len(pruned),
        'reduction': f"{(1 - len(pruned)/len(document))*100:.1f}%",
        'kept_sentences': len(indices),
        'total_sentences': len(sentences),
        'pruned_text': pruned,
        'latency': f"{latency:.2f}s",
        'indices': indices
    }

In [ ]:
question = "What is Amazon Bedrock and what models does it support?"

result = llm_prune_context(question, SAMPLE_DOCUMENT)

print(f"Question: {question}")
print(f"\nOriginal: {result['original_length']} chars, {result['total_sentences']} sentences")
print(f"Pruned: {result['pruned_length']} chars, {result['kept_sentences']} sentences")
print(f"Reduction: {result['reduction']}")
print(f"Latency: {result['latency']}")
print(f"\n--- Pruned Context ---\n{result['pruned_text']}")

## 3. Method 2: Provence Model (Specialized Pruner)

Provence is a DeBERTa-based model trained specifically for context pruning.
It's fast (1.75GB), accurate, and works out-of-the-box across domains.

Key insight from the paper: formulate pruning as **sequence labeling** - 
predict which sentences are relevant (binary mask).

In [ ]:
# Load Provence model
from transformers import AutoModel

print("Loading Provence model (this may take a moment)...")
provence = AutoModel.from_pretrained(
    "naver/provence-reranker-debertav3-v1", 
    trust_remote_code=True
)
print("✓ Provence model loaded")

In [ ]:
def provence_prune_context(question: str, document: str) -> Dict:
    """Use Provence model for efficient context pruning."""
    start = time.time()
    
    # Provence returns pruned text directly
    pruned_text = provence.process(question, document)
    
    latency = time.time() - start
    
    return {
        'original_length': len(document),
        'pruned_length': len(pruned_text),
        'reduction': f"{(1 - len(pruned_text)/len(document))*100:.1f}%",
        'pruned_text': pruned_text,
        'latency': f"{latency:.2f}s"
    }

In [ ]:
# Test Provence on the same question
question = "What is Amazon Bedrock and what models does it support?"

provence_result = provence_prune_context(question, SAMPLE_DOCUMENT)

print(f"Question: {question}")
print(f"\nOriginal: {provence_result['original_length']} chars")
print(f"Pruned: {provence_result['pruned_length']} chars")
print(f"Reduction: {provence_result['reduction']}")
print(f"Latency: {provence_result['latency']}")
print(f"\n--- Provence Pruned Context ---\n{provence_result['pruned_text']}")

## 4. Comparison: Pruned vs Full Context QA

Let's compare answer quality and latency when using:
1. Full context (no pruning)
2. LLM-pruned context
3. Provence-pruned context

In [ ]:
def answer_question(question: str, context: str, model_id: str = 'anthropic.claude-3-5-sonnet-20241022-v2:0') -> Dict:
    """Answer a question using the provided context."""
    prompt = f"""Answer the question based ONLY on the provided context.

Context:
{context}

Question: {question}

Answer:"""
    
    start = time.time()
    response = bedrock.converse(
        modelId=model_id,
        messages=[{'role': 'user', 'content': [{'text': prompt}]}],
        inferenceConfig={'temperature': 0, 'maxTokens': 1000}
    )
    latency = time.time() - start
    
    answer = response['output']['message']['content'][0]['text']
    input_tokens = response['usage']['inputTokens']
    output_tokens = response['usage']['outputTokens']
    
    return {
        'answer': answer,
        'latency': latency,
        'input_tokens': input_tokens,
        'output_tokens': output_tokens,
        'context_length': len(context)
    }

In [ ]:
question = "What is Amazon Bedrock and what models does it support?"

print("=" * 70)
print("COMPARISON: Full Context vs Pruned Context")
print("=" * 70)

# 1. Full context
print("\n📄 Method 1: FULL CONTEXT")
full_result = answer_question(question, SAMPLE_DOCUMENT)
print(f"Context: {full_result['context_length']} chars, {full_result['input_tokens']} tokens")
print(f"Latency: {full_result['latency']:.2f}s")
print(f"Answer: {full_result['answer'][:300]}...")

# 2. LLM-pruned context
print("\n✂️ Method 2: LLM-PRUNED CONTEXT")
llm_pruned = llm_prune_context(question, SAMPLE_DOCUMENT)
llm_result = answer_question(question, llm_pruned['pruned_text'])
print(f"Context: {llm_result['context_length']} chars ({llm_pruned['reduction']} reduction)")
print(f"Prune latency: {llm_pruned['latency']}, Answer latency: {llm_result['latency']:.2f}s")
print(f"Answer: {llm_result['answer'][:300]}...")

# 3. Provence-pruned context
print("\n🔬 Method 3: PROVENCE-PRUNED CONTEXT")
prov_pruned = provence_prune_context(question, SAMPLE_DOCUMENT)
prov_result = answer_question(question, prov_pruned['pruned_text'])
print(f"Context: {prov_result['context_length']} chars ({prov_pruned['reduction']} reduction)")
print(f"Prune latency: {prov_pruned['latency']}, Answer latency: {prov_result['latency']:.2f}s")
print(f"Answer: {prov_result['answer'][:300]}...")

## 5. RAG Pipeline with Context Pruning

A complete RAG pipeline that incorporates context pruning:
1. Retrieve documents
2. Prune irrelevant content
3. Generate answer

In [ ]:
class RAGWithPruning:
    """RAG pipeline with optional context pruning."""
    
    def __init__(self, documents: List[str], use_provence: bool = True):
        self.documents = documents
        self.use_provence = use_provence
        if use_provence:
            self.pruner = provence
    
    def retrieve(self, query: str, top_k: int = 3) -> List[str]:
        """Simple keyword-based retrieval (replace with vector search in production)."""
        query_words = set(query.lower().split())
        scored = []
        for doc in self.documents:
            doc_words = set(doc.lower().split())
            score = len(query_words & doc_words)
            scored.append((score, doc))
        
        scored.sort(reverse=True)
        return [doc for _, doc in scored[:top_k]]
    
    def prune(self, query: str, documents: List[str]) -> str:
        """Prune and combine retrieved documents."""
        pruned_docs = []
        for doc in documents:
            if self.use_provence:
                pruned = self.pruner.process(query, doc)
            else:
                pruned = doc
            if pruned.strip():
                pruned_docs.append(pruned)
        
        return '\n\n'.join(pruned_docs)
    
    def generate(self, query: str, context: str) -> str:
        """Generate answer using Bedrock."""
        prompt = f"""Answer the question based on the context provided.

Context:
{context}

Question: {query}

Answer:"""
        
        response = bedrock.converse(
            modelId='anthropic.claude-3-5-sonnet-20241022-v2:0',
            messages=[{'role': 'user', 'content': [{'text': prompt}]}],
            inferenceConfig={'temperature': 0, 'maxTokens': 1000}
        )
        
        return response['output']['message']['content'][0]['text']
    
    def query(self, question: str) -> Dict:
        """Full RAG pipeline: retrieve -> prune -> generate."""
        start = time.time()
        
        retrieved = self.retrieve(question)
        retrieve_time = time.time() - start
        
        prune_start = time.time()
        context = self.prune(question, retrieved)
        prune_time = time.time() - prune_start
        
        gen_start = time.time()
        answer = self.generate(question, context)
        gen_time = time.time() - gen_start
        
        return {
            'answer': answer,
            'context_length': len(context),
            'retrieved_docs': len(retrieved),
            'timings': {
                'retrieve': f"{retrieve_time:.2f}s",
                'prune': f"{prune_time:.2f}s",
                'generate': f"{gen_time:.2f}s",
                'total': f"{time.time() - start:.2f}s"
            }
        }

In [ ]:
# Create sample document corpus
DOCUMENT_CORPUS = [
    SAMPLE_DOCUMENT,
    """
    Amazon S3 (Simple Storage Service) is an object storage service offering industry-leading scalability, data availability, security, and performance. S3 is designed for 99.999999999% (11 9's) of durability.
    
    S3 storage classes include S3 Standard, S3 Intelligent-Tiering, S3 Standard-IA, S3 One Zone-IA, S3 Glacier Instant Retrieval, S3 Glacier Flexible Retrieval, and S3 Glacier Deep Archive.
    
    The Great Wall of China is a series of fortifications made of stone, brick, tamped earth, and other materials. It was built along the historical northern borders of China.
    """,
    """
    AWS Lambda is a serverless compute service that runs your code in response to events and automatically manages the underlying compute resources. Lambda supports multiple programming languages including Python, Node.js, Java, Go, Ruby, and .NET.
    
    Lambda functions can be triggered by various AWS services including API Gateway, S3, DynamoDB, SNS, and CloudWatch Events. The maximum execution time for a Lambda function is 15 minutes.
    
    Pizza is a dish of Italian origin consisting of a usually round, flat base of leavened wheat-based dough topped with tomatoes, cheese, and often various other ingredients.
    """
]

# Test the RAG pipeline
rag = RAGWithPruning(DOCUMENT_CORPUS, use_provence=True)

question = "What foundation models are available in Amazon Bedrock?"
result = rag.query(question)

print(f"Question: {question}")
print(f"\nRetrieved: {result['retrieved_docs']} documents")
print(f"Context length: {result['context_length']} chars")
print(f"Timings: {result['timings']}")
print(f"\nAnswer:\n{result['answer']}")

## 6. Bedrock-Native Pruning (No External Model)

If you can't use Provence, here's a lightweight Bedrock-only approach
using a smaller/faster model for pruning.

In [ ]:
class BedrockPruner:
    """Context pruner using only Amazon Bedrock models."""
    
    def __init__(self, pruner_model: str = 'anthropic.claude-3-haiku-20240307-v1:0'):
        """Use a fast, cheap model for pruning."""
        self.pruner_model = pruner_model
    
    def prune(self, question: str, document: str, threshold: float = 0.5) -> Dict:
        """Prune document using Bedrock model."""
        sentences = nltk.sent_tokenize(document)
        
        if len(sentences) <= 3:
            return {'pruned_text': document, 'kept': len(sentences), 'total': len(sentences)}
        
        prompt = f"""Rate each sentence's relevance to the question on a scale of 0-10.
Return ONLY a JSON object with sentence indices as keys and scores as values.

Question: {question}

Sentences:
{chr(10).join([f'{i}: {s}' for i, s in enumerate(sentences)])}

JSON scores (example format: {{"0": 8, "1": 2, "2": 9}}):"""
        
        response = bedrock.converse(
            modelId=self.pruner_model,
            messages=[{'role': 'user', 'content': [{'text': prompt}]}],
            inferenceConfig={'temperature': 0, 'maxTokens': 500}
        )
        
        content = response['output']['message']['content'][0]['text']
        if '```' in content:
            content = content.split('```')[1].replace('json', '').strip()
        
        try:
            scores = json.loads(content)
            max_score = max(float(v) for v in scores.values()) if scores else 10
            threshold_score = threshold * max_score
            
            kept_indices = [int(k) for k, v in scores.items() if float(v) >= threshold_score]
            kept_indices.sort()
            
            pruned = ' '.join([sentences[i] for i in kept_indices if i < len(sentences)])
        except:
            pruned = document
            kept_indices = list(range(len(sentences)))
        
        return {
            'pruned_text': pruned,
            'kept': len(kept_indices),
            'total': len(sentences),
            'reduction': f"{(1 - len(pruned)/len(document))*100:.1f}%"
        }

In [ ]:
# Test Bedrock-native pruning
bedrock_pruner = BedrockPruner()

question = "What is Amazon Bedrock?"
result = bedrock_pruner.prune(question, SAMPLE_DOCUMENT)

print(f"Question: {question}")
print(f"Kept {result['kept']}/{result['total']} sentences ({result['reduction']} reduction)")
print(f"\n--- Pruned Context ---\n{result['pruned_text']}")

## 7. Measuring Pruning Quality

How do we know if pruning is working well? Key metrics:
1. **Compression ratio**: How much did we reduce?
2. **Answer quality**: Is the answer still correct?
3. **Latency savings**: How much faster is generation?

In [ ]:
def evaluate_pruning(question: str, document: str, ground_truth: str) -> Dict:
    """Evaluate pruning quality across multiple methods."""
    results = {}
    
    # Baseline: Full context
    print("Testing full context...")
    full_start = time.time()
    full_answer = answer_question(question, document)
    results['full_context'] = {
        'context_chars': len(document),
        'latency': time.time() - full_start,
        'answer': full_answer['answer'],
        'input_tokens': full_answer['input_tokens']
    }
    
    # LLM pruning
    print("Testing LLM pruning...")
    llm_start = time.time()
    llm_pruned = llm_prune_context(question, document)
    llm_answer = answer_question(question, llm_pruned['pruned_text'])
    results['llm_pruned'] = {
        'context_chars': len(llm_pruned['pruned_text']),
        'latency': time.time() - llm_start,
        'answer': llm_answer['answer'],
        'input_tokens': llm_answer['input_tokens'],
        'reduction': llm_pruned['reduction']
    }
    
    # Provence pruning
    print("Testing Provence pruning...")
    prov_start = time.time()
    prov_pruned = provence_prune_context(question, document)
    prov_answer = answer_question(question, prov_pruned['pruned_text'])
    results['provence_pruned'] = {
        'context_chars': len(prov_pruned['pruned_text']),
        'latency': time.time() - prov_start,
        'answer': prov_answer['answer'],
        'input_tokens': prov_answer['input_tokens'],
        'reduction': prov_pruned['reduction']
    }
    
    # Simple relevance check using LLM
    print("Evaluating answer quality...")
    for method, data in results.items():
        eval_prompt = f"""Rate how well this answer addresses the question (1-10).
Also check if it contains the key information: {ground_truth}

Question: {question}
Answer: {data['answer']}

Return JSON: {{"score": X, "contains_key_info": true/false}}"""
        
        eval_response = bedrock.converse(
            modelId='anthropic.claude-3-haiku-20240307-v1:0',
            messages=[{'role': 'user', 'content': [{'text': eval_prompt}]}],
            inferenceConfig={'temperature': 0, 'maxTokens': 100}
        )
        
        try:
            eval_content = eval_response['output']['message']['content'][0]['text']
            if '```' in eval_content:
                eval_content = eval_content.split('```')[1].replace('json', '').strip()
            eval_result = json.loads(eval_content)
            data['quality_score'] = eval_result.get('score', 0)
            data['contains_key_info'] = eval_result.get('contains_key_info', False)
        except:
            data['quality_score'] = 'N/A'
            data['contains_key_info'] = 'N/A'
    
    return results

In [ ]:
# Run evaluation
question = "What models does Amazon Bedrock support?"
ground_truth = "Claude, Titan, Llama, Command, Jurassic"

eval_results = evaluate_pruning(question, SAMPLE_DOCUMENT, ground_truth)

print("\n" + "=" * 70)
print("PRUNING EVALUATION RESULTS")
print("=" * 70)

for method, data in eval_results.items():
    print(f"\n📊 {method.upper()}")
    print(f"   Context: {data['context_chars']} chars, {data['input_tokens']} tokens")
    print(f"   Latency: {data['latency']:.2f}s")
    if 'reduction' in data:
        print(f"   Reduction: {data['reduction']}")
    print(f"   Quality Score: {data['quality_score']}/10")
    print(f"   Contains Key Info: {data['contains_key_info']}")

## 8. Context Summarization (Alternative to Pruning)

For very long contexts or conversation histories, **summarization** can be more effective than pruning.
The Pokémon-playing Gemini agent found that contexts beyond 100k tokens caused the model to repeat actions rather than synthesize new plans.

Key insight: Summarization preserves semantic meaning while dramatically reducing tokens.

In [ ]:
def summarize_context(document: str, question: str = None, max_length: int = 500) -> Dict:
    """Summarize context while preserving information relevant to the question."""
    
    if question:
        prompt = f"""Summarize the following document, focusing on information relevant to this question: {question}

Document:
{document}

Provide a concise summary (max {max_length} characters) that preserves key facts:"""
    else:
        prompt = f"""Summarize the following document, preserving all key facts and information.

Document:
{document}

Provide a concise summary (max {max_length} characters):"""
    
    start = time.time()
    response = bedrock.converse(
        modelId='anthropic.claude-3-haiku-20240307-v1:0',
        messages=[{'role': 'user', 'content': [{'text': prompt}]}],
        inferenceConfig={'temperature': 0, 'maxTokens': 1000}
    )
    latency = time.time() - start
    
    summary = response['output']['message']['content'][0]['text']
    
    return {
        'summary': summary,
        'original_length': len(document),
        'summary_length': len(summary),
        'reduction': f"{(1 - len(summary)/len(document))*100:.1f}%",
        'latency': f"{latency:.2f}s"
    }

In [ ]:
# Test summarization
question = "What is Amazon Bedrock and what models does it support?"

summary_result = summarize_context(SAMPLE_DOCUMENT, question)

print(f"Question: {question}")
print(f"\nOriginal: {summary_result['original_length']} chars")
print(f"Summary: {summary_result['summary_length']} chars")
print(f"Reduction: {summary_result['reduction']}")
print(f"Latency: {summary_result['latency']}")
print(f"\n--- Summary ---\n{summary_result['summary']}")

# Compare answer quality with summarized context
print("\n" + "="*50)
print("Answer using summarized context:")
summary_answer = answer_question(question, summary_result['summary'])
print(summary_answer['answer'])

### Pruning vs Summarization

| Aspect | Pruning | Summarization |
|--------|---------|---------------|
| **Preserves** | Original sentences | Semantic meaning |
| **Best for** | Removing irrelevant content | Compressing long histories |
| **Speed** | Fast (Provence) | Slower (LLM call) |
| **Risk** | May lose context | May lose details |
| **Use case** | RAG, document QA | Long conversations, agents |

## 9. When to Use Context Pruning

Context pruning is most valuable when:

| Scenario | Benefit |
|----------|---------|
| Long retrieved documents | Reduce tokens, save cost |
| Multiple retrieved passages | Remove redundancy |
| Noisy retrieval results | Filter irrelevant content |
| Latency-sensitive apps | Faster generation |
| Edge/mobile deployment | Lower compute requirements |

### Trade-offs

| Method | Speed | Quality | Cost |
|--------|-------|---------|------|
| No pruning | Fast | Baseline | High tokens |
| LLM pruning | Slow | High | Extra LLM call |
| Provence | Very fast | High | Model hosting |
| Bedrock Haiku | Medium | Good | Cheap LLM call |

## 10. Best Practices

1. **Prune before generation, not after** - Save tokens and improve focus
2. **Use specialized pruners for high volume** - Provence is 10-100x faster than LLM pruning
3. **Preserve sentence order** - Maintains coherence and context flow
4. **Set appropriate thresholds** - Too aggressive = lost information, too lenient = no benefit
5. **Combine with other techniques**:
   - Context Quarantine (isolate agent contexts)
   - Tool Loadout (prune tool definitions)
   - Context Summarization (for very long histories)

## Key Takeaways

✅ **Context pruning removes irrelevant information** before LLM generation  
✅ **Provence** is a fast, accurate, domain-agnostic pruner (ICLR 2025)  
✅ **LLM-based pruning** works but adds latency and cost  
✅ **Pruning improves quality** by reducing context confusion  
✅ **Pruning reduces cost** by lowering token count  

### References

- [How to Fix Your Context](https://www.dbreunig.com/2025/06/26/how-to-fix-your-context.html) - Drew Breunig
- [Provence: Efficient and Robust Context Pruning](https://arxiv.org/abs/2501.16214) - ICLR 2025
- [Less is More: Optimizing Function Calling](https://arxiv.org/abs/2411.15399) - Tool pruning
- [RAG MCP](https://arxiv.org/abs/2505.03275) - Tool selection via RAG